In [ ]:
!pip install pyspark

In [ ]:
!pip install tweepy

In [ ]:
!pip install findspark

In [1]:
# May cause deprecation warnings, safe to ignore, they aren't errors
import findspark
findspark.init()
from pyspark import SparkContext, SparkConf
from pyspark.streaming import StreamingContext
from pyspark.sql import SQLContext, SparkSession, Row
from pyspark.sql.functions import desc
from pyspark.rdd import RDD
import datetime

def getSparkSessionInstance(sparkConf: SparkConf) -> SparkSession:
    if ('sparkSessionSingletonInstance' not in globals()):
        globals()['sparkSessionSingletonInstance'] = SparkSession\
            .builder\
            .config(conf=sparkConf)\
            .getOrCreate()
    return globals()['sparkSessionSingletonInstance']


# Can only run this once. restart your kernel for any errors.
sc = SparkContext(appName = "Tweeter")
ssc = StreamingContext(sc, 10 )
sqlContext = SQLContext(sc)
socket_stream = ssc.socketTextStream("127.0.0.1", 5554)
lines = socket_stream.window( 20 )
from collections import namedtuple
fields = ("tag", "count" )
Tweet = namedtuple( 'Tweet', fields )
words = lines.flatMap(lambda line: line.split(" "))

# Use Parenthesis for multiple lines or use \.
"""( lines.flatMap( lambda text: text.split( " " ) ) #Splits to a list
  .filter( lambda word: word.lower().startswith("#") ) # Checks for hashtag calls
  .map( lambda word: ( word.lower(), 1 ) ) # Lower cases the word
  .reduceByKey( lambda a, b: a + b ) # Reduces
  .map( lambda rec: Tweet( rec[0], rec[1] ) ) # Stores in a Tweet Object
  .foreachRDD( lambda rdd: rdd.toDF().sort( ("count") ) # Sorts Them in a DF
  .limit(10).createOrReplaceTempView ("tweets") ) ) # Registers to a table
"""
def process(time: datetime.datetime, rdd) -> None:
    print("========= %s =========" % str(time))

    try:
        # Get the singleton instance of SparkSession
        spark = getSparkSessionInstance(rdd.context.getConf())

        # Convert RDD[String] to RDD[Row] to DataFrame
        rowRdd = rdd.map(lambda w: Row(word=w))
        wordsDataFrame = spark.createDataFrame(rowRdd)

        # Creates a temporary view using the DataFrame.
        wordsDataFrame.createOrReplaceTempView("words")

        # Do word count on table using SQL and print it
        wordCountsDataFrame = \
            spark.sql("select word, count(*) as total from words group by word")
        wordCountsDataFrame.show()
    except BaseException:
        pass

words.foreachRDD(process)
ssc.start()
ssc.awaitTermination()

C:\Spark\python\pyspark\sql\context.py:77: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


========= 2022-11-14 19:46:10 =========
+--------------------+-----+
|                word|total|
+--------------------+-----+
|               #Math|    1|
|                 not|    1|
|                you!|    1|
|                 #ML|    1|
|             #Python|    1|
|             already|    1|
|              You're|    2|
|             talking|    1|
|                #IoT|    1|
|                  is|    2|
|https://t.co/Jmdc...|    1|
|                   @|    1|
|          listening.|    1|
|                data|    1|
|        #Engineering|    1|
|            probably|    1|
|https://t.co/WMtX...|    1|
|          Consulting|    1|
|             #Pandas|    1|
|        #DataScience|    1|
+--------------------+-----+
only showing top 20 rows

========= 2022-11-14 19:46:20 =========
+--------------------+-----+
|                word|total|
+--------------------+-----+
|               #Math|    1|
|                 not|    1|
|                you!|    1|
|                 #ML|  

In [4]:
lines.pprint()

Py4JJavaError: An error occurred while calling z:org.apache.spark.streaming.api.python.PythonDStream.callForeachRDD.
: java.lang.IllegalStateException: Adding new inputs, transformations, and output operations after starting a context is not supported
	at org.apache.spark.streaming.dstream.DStream.validateAtInit(DStream.scala:226)
	at org.apache.spark.streaming.dstream.DStream.<init>(DStream.scala:67)
	at org.apache.spark.streaming.dstream.ForEachDStream.<init>(ForEachDStream.scala:39)
	at org.apache.spark.streaming.dstream.DStream.foreachRDD(DStream.scala:655)
	at org.apache.spark.streaming.dstream.DStream.$anonfun$foreachRDD$3(DStream.scala:640)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:112)
	at org.apache.spark.SparkContext.withScope(SparkContext.scala:792)
	at org.apache.spark.stre

In [4]:
import time
from IPython import display
import matplotlib.pyplot as plt
import seaborn as sns
# Only works for Jupyter Notebooks!
%matplotlib inline 

In [5]:
count = 0
while count < 10:
    
    time.sleep( 3 )
    top_10_tweets = sqlContext.sql( 'Select tag, count from tweets' )
    top_10_df = top_10_tweets.toPandas()
    display.clear_output(wait=True)
    plt.figure( figsize = ( 10, 8 ) )
#     sns.barplot(x='count',y='land_cover_specific', data=df, palette='Spectral')
    sns.barplot( x="count", y="tag", data=top_10_df)
    plt.show()
    count = count + 1

AnalysisException: Table or view not found: tweets; line 1 pos 23;
'Project ['tag, 'count]
+- 'UnresolvedRelation [tweets], [], false


In [ ]:
ssc.stop()